In [7]:
import pandas as pd
import xgboost as xgb
from sklearn.preprocessing import LabelEncoder
import numpy as np

C:\Users\bhanu\Anaconda3\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [19]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
import os
os.environ['OMP_NUM_THREADS'] = '4'
import gc

In [13]:
cd Downloads/

C:\Users\bhanu\Downloads


In [21]:
dtypes = {
        'ip'            : 'uint32',
        'app'           : 'uint16',
        'device'        : 'uint16',
        'os'            : 'uint16',
        'channel'       : 'uint16',
        'is_attributed' : 'uint8',
        'click_id'      : 'uint32'
        }

In [22]:
train_df = pd.read_csv("train_talking_data.csv", dtype=dtypes, skiprows = range(1, 131886954), usecols=['ip','app','device','os', 'channel', 'click_time', 'is_attributed'])

In [23]:
train_df.shape

(53016937, 7)

In [24]:
train_df.head(5)

,ip,app,device,os,channel,click_time,is_attributed
0,201143,11,1,13,487,2017-11-09 00:00:00,0
1,34684,2,1,13,469,2017-11-09 00:00:00,0
2,207368,26,1,19,477,2017-11-09 00:00:00,0
3,110176,18,1,8,121,2017-11-09 00:00:00,0
4,109644,12,1,19,265,2017-11-09 00:00:00,0


In [25]:
train_df.describe()

,ip,app,device,os,channel,is_attributed
count,5.301694e+07,5.301694e+07,5.301694e+07,5.301694e+07,5.301694e+07,5.301694e+07
mean,9.548046e+04,1.144590e+01,1.246275e+01,2.113352e+01,2.644607e+02,2.526872e-03
std,8.136274e+04,1.271698e+01,2.067801e+02,4.927935e+01,1.325907e+02,5.020445e-02
min,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,3.983400e+04,3.000000e+00,1.000000e+00,1.300000e+01,1.370000e+02,0.000000e+00
50%,7.880500e+04,9.000000e+00,1.000000e+00,1.800000e+01,2.450000e+02,0.000000e+00
75%,1.161870e+05,1.500000e+01,1.000000e+00,1.900000e+01,3.790000e+02,0.000000e+00
max,3.647780e+05,7.680000e+02,4.227000e+03,9.560000e+02,5.000000e+02,1.000000e+00


In [26]:
test_df =pd.read_csv("test.csv", dtype=dtypes, usecols=['ip','app','device','os', 'channel', 'click_time', 'click_id'])

In [27]:
len_train = len(train_df)
train_df = train_df.append(test_df)

In [28]:
train_df.shape

(71807406, 8)

In [29]:
#delete and colelct garbage
del test_df; gc.collect()

439

In [31]:
print('hour, day, wday....')
train_df['hour'] = pd.to_datetime(train_df.click_time).dt.hour.astype('uint8')
train_df['day'] = pd.to_datetime(train_df.click_time).dt.day.astype('uint8')
train_df['wday']  = pd.to_datetime(train_df.click_time).dt.dayofweek.astype('uint8')

hour, day, wday....


In [32]:
train_df.head(5)

,app,channel,click_id,click_time,device,ip,is_attributed,os,hour,day,wday
0,11,487,NaN,2017-11-09 00:00:00,1,201143,0.0,13,0,9,3
1,2,469,NaN,2017-11-09 00:00:00,1,34684,0.0,13,0,9,3
2,26,477,NaN,2017-11-09 00:00:00,1,207368,0.0,19,0,9,3
3,18,121,NaN,2017-11-09 00:00:00,1,110176,0.0,8,0,9,3
4,12,265,NaN,2017-11-09 00:00:00,1,109644,0.0,19,0,9,3


In [36]:
print('grouping by ip alone....')
gp = train_df[['ip','channel']].groupby(by=['ip'])[['channel']].count().reset_index().rename(index=str, columns={'channel': 'ipcount'})
train_df = train_df.merge(gp, on=['ip'], how='left')
del gp; gc.collect()

grouping by ip alone....


152

In [37]:
print('grouping by ip-day-hour combination....')
gp = train_df[['ip','day','hour','channel']].groupby(by=['ip','day','hour'])[['channel']].count().reset_index().rename(index=str, columns={'channel': 'qty'})
train_df = train_df.merge(gp, on=['ip','day','hour'], how='left')
del gp; gc.collect()

grouping by ip-day-hour combination....


117

In [40]:
print('grouping by ip-day-hour combination....')
gp = train_df[['ip','day','hour','channel']].groupby(by=['ip','day','hour'])[['channel']].count().reset_index().rename(index=str, columns={'channel': 'qty'})
train_df = train_df.merge(gp, on=['ip','day','hour'], how='left')
del gp; gc.collect()

grouping by ip-day-hour combination....


221

In [41]:
print('group by ip-app combination....')
gp = train_df[['ip','app', 'channel']].groupby(by=['ip', 'app'])[['channel']].count().reset_index().rename(index=str, columns={'channel': 'ip_app_count'})
train_df = train_df.merge(gp, on=['ip','app'], how='left')
del gp; gc.collect()
print('group by ip-app-os combination....')
gp = train_df[['ip','app', 'os', 'channel']].groupby(by=['ip', 'app', 'os'])[['channel']].count().reset_index().rename(index=str, columns={'channel': 'ip_app_os_count'})
train_df = train_df.merge(gp, on=['ip','app', 'os'], how='left')
del gp; gc.collect()

group by ip-app combination....
group by ip-app-os combination....


117

In [42]:
print("vars and data type....")
train_df['ipcount'] = train_df['qty'].astype('uint32')
train_df['qty'] = train_df['qty'].astype('uint16')
train_df['ip_app_count'] = train_df['ip_app_count'].astype('uint16')
train_df['ip_app_os_count'] = train_df['ip_app_os_count'].astype('uint16')

vars and data type....


In [43]:
print("label encoding....")
from sklearn.preprocessing import LabelEncoder
train_df[['app','device','os', 'channel', 'hour', 'day', 'wday']].apply(LabelEncoder().fit_transform)
print ('final part of preparation....')
test_df = train_df[len_train:]
train_df = train_df[:len_train]
y_train = train_df['is_attributed'].values
train_df.drop(['click_id', 'click_time','ip','is_attributed'],1,inplace=True)

label encoding....
final part of preparation....


In [44]:
print ('neural network....')
from keras.layers import Input, Embedding, Dense, Flatten, Dropout, concatenate
from keras.layers import BatchNormalization, SpatialDropout1D, GaussianDropout
from keras.callbacks import Callback
from keras.models import Model
from keras.optimizers import Adam

neural network....


Using TensorFlow backend.


In [45]:
max_app = np.max([train_df['app'].max(), test_df['app'].max()])+1
max_ch = np.max([train_df['channel'].max(), test_df['channel'].max()])+1
max_dev = np.max([train_df['device'].max(), test_df['device'].max()])+1
max_os = np.max([train_df['os'].max(), test_df['os'].max()])+1
max_h = np.max([train_df['hour'].max(), test_df['hour'].max()])+1
max_qty = np.max([train_df['qty'].max(), test_df['qty'].max()])+1
max_ipcount = np.max([train_df['ipcount'].max(), test_df['ipcount'].max()])+1
max_c1 = np.max([train_df['ip_app_count'].max(), test_df['ip_app_count'].max()])+1
max_c2 = np.max([train_df['ip_app_os_count'].max(), test_df['ip_app_os_count'].max()])+1

In [46]:
def get_keras_data(dataset):
    X = {
        'app': np.array(dataset.app),
        'ch': np.array(dataset.channel),
        'dev': np.array(dataset.device),
        'os': np.array(dataset.os),
        'h': np.array(dataset.hour),
        'qty': np.array(dataset.qty),
        'ipcount': np.array(dataset.ipcount),
        'c1': np.array(dataset.ip_app_count),
        'c2': np.array(dataset.ip_app_os_count)
    }
    return X
train_df = get_keras_data(train_df)


In [ ]:
emb_n = 50
dense_n1 = 1000
dense_n2 = 1000
dense_n3 = 200
dense_n4 = 40
in_app = Input(shape=[1], name = 'app')
emb_app = Embedding(max_app, emb_n)(in_app)
in_ch = Input(shape=[1], name = 'ch')
emb_ch = Embedding(max_ch, emb_n)(in_ch)
in_dev = Input(shape=[1], name = 'dev')
emb_dev = Embedding(max_dev, emb_n)(in_dev)
in_os = Input(shape=[1], name = 'os')
emb_os = Embedding(max_os, emb_n)(in_os)
in_h = Input(shape=[1], name = 'h')
emb_h = Embedding(max_h, emb_n)(in_h) 
in_qty = Input(shape=[1], name = 'qty')
emb_qty = Embedding(max_qty, emb_n)(in_qty) 
in_ipcount = Input(shape=[1], name = 'ipcount')
emb_ipcount = Embedding(max_ipcount, 2*emb_n)(in_ipcount) 
in_c1 = Input(shape=[1], name = 'c1')
emb_c1 = Embedding(max_c1, emb_n)(in_c1) 
in_c2 = Input(shape=[1], name = 'c2')
emb_c2 = Embedding(max_c2, emb_n)(in_c2) 
fe = concatenate([(emb_app), (emb_ch), (emb_dev), (emb_os), (emb_h), 
                  (emb_ipcount), (emb_qty), (emb_c1), (emb_c2)])
s_dout = SpatialDropout1D(0.2)(fe)